## 0. Cài đặt và Setup môi trường

In [ ]:
# Gỡ bỏ phiên bản hiện tại để tránh lỗi
!pip uninstall -y numpy opencv-python opencv-python-headless stardist csbdeep

# Cài đặt phiên bản tương thích (NumPy 1.x cho Colab)
!pip install "numpy<2.0.0" "opencv-python-headless<4.10"
!pip install stardist csbdeep

print("✅ Đã cài đặt xong. VUI LÒNG KHỞI ĐỘNG LẠI RUNTIME!")
print("   Runtime > Restart runtime")

In [ ]:
# Patch cho Python 3.12 compatibility
import configparser

if not hasattr(configparser, 'SafeConfigParser'):
    configparser.SafeConfigParser = configparser.ConfigParser
    print("✅ Đã vá lỗi SafeConfigParser thành công!")

## 1. Mount Google Drive và Setup

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Tạo thư mục làm việc
work_dir = '/content/stardist_project'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"✅ Working directory: {os.getcwd()}")

## 2. Upload Dataset

**CÁCH 1: Upload từ máy tính**

In [ ]:
from google.colab import files
import zipfile

print("📤 Chọn file my_dataset.zip để upload")
uploaded = files.upload()

# Giải nén
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✅ Extracted!")

!ls -la

**CÁCH 2: Copy từ Google Drive (KHUYẾN NGHỊ)**

In [ ]:
import shutil
import subprocess

# Đường dẫn file trên Google Drive
drive_dataset_path = '/content/drive/MyDrive/my_dataset.rar'
local_filename = 'my_dataset.rar'

if os.path.exists(drive_dataset_path):
    print("Copying from Google Drive...")
    shutil.copy2(drive_dataset_path, local_filename)
    
    # Cài unrar và giải nén
    !apt-get install unrar -qq
    print("Extracting RAR file...")
    
    result = subprocess.run(['unrar', 'x', '-y', local_filename],
                            capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Dataset copied and extracted successfully!")
    else:
        print("❌ Lỗi khi giải nén:", result.stderr)
else:
    print(f"⚠️ Không tìm thấy file tại: {drive_dataset_path}")

## 3. Import Libraries và Check GPU

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

from csbdeep.utils import normalize
from stardist import fill_label_holes, random_label_cmap, calculate_extents
from stardist.matching import matching_dataset
from stardist.models import Config2D, StarDist2D

import tensorflow as tf

np.random.seed(42)
lbl_cmap = random_label_cmap()

print(f"NumPy version: {np.__version__}")
print(f"TensorFlow version: {tf.__version__}")

if len(tf.config.list_physical_devices('GPU')) > 0:
    print("\n🚀 GPU detected! Training will be FAST.")
    use_gpu = True
else:
    print("\n⚠️ No GPU! Hãy bật GPU: Runtime > Change runtime type > GPU")
    use_gpu = False

## 4. Load Data

In [ ]:
data_dir = Path('my_dataset')

X_train_files = sorted(glob(str(data_dir / 'train' / 'images' / '*')))
Y_train_files = sorted(glob(str(data_dir / 'train' / 'masks' / '*')))
X_val_files = sorted(glob(str(data_dir / 'val' / 'images' / '*')))
Y_val_files = sorted(glob(str(data_dir / 'val' / 'masks' / '*')))

print(f"📊 Dataset size:")
print(f"   Training: {len(X_train_files)} images")
print(f"   Validation: {len(X_val_files)} images")
print(f"   Total: {len(X_train_files) + len(X_val_files)} images")

# Kiểm tra
assert len(X_train_files) > 0, "⚠️ No training images found!"
assert len(X_train_files) == len(Y_train_files), "⚠️ Mismatch in train images/masks!"
assert len(X_val_files) == len(Y_val_files), "⚠️ Mismatch in val images/masks!"

# Đánh giá dataset size
total_images = len(X_train_files) + len(X_val_files)
if total_images >= 150:
    print("\n✅ Dataset size EXCELLENT! Expected AP > 0.85")
elif total_images >= 100:
    print("\n✅ Dataset size GOOD! Expected AP 0.75-0.85")
elif total_images >= 50:
    print("\n⚠️ Dataset size OK. Expected AP 0.65-0.75")
else:
    print("\n⚠️ Dataset size SMALL! Expected AP < 0.70")

# Load images
print("\n📥 Loading images...")
X_train = [np.array(Image.open(f)) for f in tqdm(X_train_files, desc="Train images")]
Y_train = [np.array(Image.open(f)) for f in tqdm(Y_train_files, desc="Train masks")]
X_val = [np.array(Image.open(f)) for f in tqdm(X_val_files, desc="Val images")]
Y_val = [np.array(Image.open(f)) for f in tqdm(Y_val_files, desc="Val masks")]

print("\n✅ Data loaded!")

## 5. Preprocessing

In [ ]:
n_channel = 1 if X_train[0].ndim == 2 else X_train[0].shape[-1]
print(f"Channels: {n_channel}")

axis_norm = (0,1)
X_train = [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_train, desc="Normalize train")]
X_val = [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_val, desc="Normalize val")]

Y_train = [fill_label_holes(y) for y in tqdm(Y_train, desc="Fill holes train")]
Y_val = [fill_label_holes(y) for y in tqdm(Y_val, desc="Fill holes val")]

print("✅ Preprocessing done!")

## 6. 🔥 STRONG AUGMENTATION - CẢI TIẾN!

**Các kỹ thuật augmentation:**
- Rotation: 0-360°
- Flip: horizontal + vertical
- Elastic deformation: biến dạng đàn hồi (quan trọng cho cells!)
- Brightness: ±30%
- Contrast: ±20%
- Gaussian noise: nhiễu nhẹ

In [ ]:
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage.transform import rotate

def augmenter_strong(x, y):
    """
    Strong augmentation for StarDist training
    """
    # 1. Random rotation (0-360 degrees)
    if np.random.rand() > 0.5:
        angle = np.random.uniform(0, 360)
        x = rotate(x, angle, mode='reflect', preserve_range=True)
        y = rotate(y, angle, order=0, mode='reflect', preserve_range=True).astype(y.dtype)
    
    # 2. Random flip
    if np.random.rand() > 0.5:
        x = np.flip(x, axis=0)
        y = np.flip(y, axis=0)
    if np.random.rand() > 0.5:
        x = np.flip(x, axis=1)
        y = np.flip(y, axis=1)
    
    # 3. Elastic deformation
    if np.random.rand() > 0.5:
        alpha = 30
        sigma = 5
        
        shape = x.shape[:2]
        dx = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma) * alpha
        dy = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma) * alpha
        
        x_coords = np.arange(shape[0])[:, None] + dx
        y_coords = np.arange(shape[1])[None, :] + dy
        
        indices = np.array([x_coords, y_coords])
        
        if x.ndim == 3:
            x = np.stack([map_coordinates(x[..., i], indices, order=1, mode='reflect') 
                          for i in range(x.shape[-1])], axis=-1)
        else:
            x = map_coordinates(x, indices, order=1, mode='reflect')
        
        y = map_coordinates(y, indices, order=0, mode='reflect').astype(y.dtype)
    
    # 4. Brightness adjustment
    if np.random.rand() > 0.5:
        factor = np.random.uniform(0.7, 1.3)
        x = np.clip(x * factor, 0, 1)
    
    # 5. Contrast adjustment
    if np.random.rand() > 0.5:
        factor = np.random.uniform(0.8, 1.2)
        mean = x.mean()
        x = np.clip((x - mean) * factor + mean, 0, 1)
    
    # 6. Gaussian noise
    if np.random.rand() > 0.7:
        noise = np.random.normal(0, 0.02, x.shape)
        x = np.clip(x + noise, 0, 1)
    
    return x, y

print("✅ Strong augmentation function created!")
print("\n📊 Augmentation details:")
print("   - Rotation: 0-360°")
print("   - Flip: H + V")
print("   - Elastic: α=30, σ=5")
print("   - Brightness: ±30%")
print("   - Contrast: ±20%")
print("   - Gaussian noise: σ=0.02")

## 7. Visualize Augmentation

In [ ]:
test_idx = 0
test_x = X_train[test_idx]
test_y = Y_train[test_idx]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Original
axes[0, 0].imshow(test_x if test_x.ndim == 3 else test_x, cmap='gray' if test_x.ndim == 2 else None)
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(test_y, cmap=lbl_cmap)
axes[0, 1].set_title('Original Mask')
axes[0, 1].axis('off')

# Augmented examples
for i in range(3):
    aug_x, aug_y = augmenter_strong(test_x.copy(), test_y.copy())
    
    col = (i + 1) * 2 if i < 1 else i
    row = 0 if i < 1 else 1
    
    axes[row, col].imshow(aug_x if aug_x.ndim == 3 else aug_x, cmap='gray' if aug_x.ndim == 2 else None)
    axes[row, col].set_title(f'Augmented {i+1}')
    axes[row, col].axis('off')
    
    axes[row, col+1].imshow(aug_y, cmap=lbl_cmap)
    axes[row, col+1].set_title(f'Augmented Mask {i+1}')
    axes[row, col+1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Augmentation looks good!")

## 8. 🔧 CONFIG TỐI ƯU - Auto-tuned!

In [ ]:
print("🔧 Configuring optimized model...\n")

n_train = len(X_train)
n_val = len(X_val)

# Auto-adjust parameters based on dataset size
if n_train >= 100:
    train_epochs = 150
    steps_per_epoch = 200
    print("📊 Large dataset detected (≥100 images)")
    print(f"   → epochs={train_epochs}, steps_per_epoch={steps_per_epoch}")
elif n_train >= 50:
    train_epochs = 200
    steps_per_epoch = 150
    print("📊 Medium dataset detected (50-100 images)")
    print(f"   → epochs={train_epochs}, steps_per_epoch={steps_per_epoch}")
else:
    train_epochs = 250
    steps_per_epoch = 100
    print("📊 Small dataset detected (<50 images)")
    print(f"   → epochs={train_epochs}, steps_per_epoch={steps_per_epoch}")

# Model configuration
img_size = X_train[0].shape[:2]
train_patch_size = (256, 256) if min(img_size) < 512 else (512, 512)

print(f"\n🎯 Model configuration:")
print(f"   Image size: {img_size}")
print(f"   Patch size: {train_patch_size}")
print(f"   Channels: {n_channel}")
print(f"   GPU: {use_gpu}")

conf = Config2D(
    n_rays=64,
    grid=(2, 2),
    use_gpu=use_gpu,
    n_channel_in=n_channel,
    train_patch_size=train_patch_size,
    
    # Training parameters - OPTIMIZED!
    train_epochs=train_epochs,
    train_steps_per_epoch=steps_per_epoch,
    train_learning_rate=0.0003,
    train_batch_size=4,
    train_reduce_lr={'factor': 0.5, 'patience': 10},
    
    # Regularization
    train_background_reg=0.0001,
    train_foreground_only=0.9,
)

print(f"\n✅ Configuration created!")
print(conf)

## 9. Create Model

In [ ]:
model_name = 'stardist_my_data_v2_improved'
model_basedir = 'models'

print(f"Creating model: {model_name}...")
model = StarDist2D(conf, name=model_name, basedir=model_basedir)

print(f"\n✅ Model '{model_name}' created!")
print(f"   Location: {model_basedir}/{model_name}/")
print(f"   Total parameters: {model.keras_model.count_params():,}")

## 10. 🚀 TRAINING!

**Expected training time:**
- With GPU: 30-60 minutes
- Without GPU: 2-4 hours

In [ ]:
print("="*60)
print("🚀 STARTING TRAINING WITH STRONG AUGMENTATION")
print("="*60)
print(f"\n📊 Dataset: {n_train} train, {n_val} val")
print(f"⚙️ Config: {train_epochs} epochs × {steps_per_epoch} steps")
print(f"🎨 Augmentation: Strong (rotation+flip+elastic+brightness+contrast+noise)")
print(f"\n⏱️ Estimated time: {'30-60 min' if use_gpu else '2-4 hours'}")
print("="*60 + "\n")

# Training!
history = model.train(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    augmenter=augmenter_strong,
    epochs=conf.train_epochs,
    steps_per_epoch=conf.train_steps_per_epoch,
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETED!")
print("="*60)

## 11. Training History

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Loss
ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# Learning rate
if 'lr' in history.history:
    ax2.plot(history.history['lr'], linewidth=2, color='green')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Learning Rate', fontsize=12)
    ax2.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    ax2.set_yscale('log')
    ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{model_basedir}/{model_name}/training_history.png', dpi=150)
plt.show()

best_epoch = np.argmin(history.history['val_loss']) + 1
best_val_loss = min(history.history['val_loss'])
print(f"Best epoch: {best_epoch}/{len(history.history['val_loss'])}")
print(f"Best validation loss: {best_val_loss:.4f}")

## 12. Optimize Thresholds

In [ ]:
print("🎯 Optimizing thresholds...\n")

model.optimize_thresholds(X_val, Y_val)

print(f"\n✅ Optimized thresholds:")
print(f"   prob_thresh = {model.thresholds.prob:.3f}")
print(f"   nms_thresh = {model.thresholds.nms:.3f}")

## 13. 📊 EVALUATION

**AP (Average Precision) targets:**
- ⭐⭐⭐ Excellent: AP > 0.85
- ⭐⭐ Very Good: AP 0.75-0.85
- ⭐ Good: AP 0.65-0.75
- ⚠️ Needs improvement: AP < 0.65

In [ ]:
print("="*60)
print("📊 EVALUATING MODEL PERFORMANCE")
print("="*60)

# Predict on validation set
print("\nPredicting on validation set...")
Y_val_pred = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(X_val, desc="Predicting")]

# Compute metrics
print("\nComputing metrics...")
stats = [matching_dataset(Y_val, Y_val_pred, thresh=t, show_progress=True) 
         for t in tqdm([0.5, 0.6, 0.7, 0.8, 0.9], desc="Computing metrics")]

print("\n" + "="*60)
print("VALIDATION RESULTS")
print("="*60)
for i, (thresh, stat) in enumerate(zip([0.5, 0.6, 0.7, 0.8, 0.9], stats)):
    print(f"IoU {thresh}: AP = {stat.mean_matched_score:.3f}")

ap_50 = stats[0].mean_matched_score
print(f"\n🎯 OVERALL SCORE (AP@0.5): {ap_50:.3f}")
print("="*60)

# Performance assessment
if ap_50 >= 0.85:
    print("\n🎉🎉🎉 EXCELLENT! AP ≥ 0.85")
    print("✅ Model is ready for production!")
elif ap_50 >= 0.75:
    print("\n✅✅ VERY GOOD! AP 0.75-0.85")
    print("✅ Model performance is strong.")
elif ap_50 >= 0.65:
    print("\n✅ GOOD! AP 0.65-0.75")
    print("⚠️ Model is acceptable but can be improved.")
else:
    print("\n⚠️ NEEDS IMPROVEMENT! AP < 0.65")
    print("💡 Consider adding more training data")

## 14. Visualize Results

In [ ]:
n_samples = min(4, len(X_val))
sample_indices = np.random.choice(len(X_val), n_samples, replace=False)

fig, axes = plt.subplots(n_samples, 4, figsize=(16, 4*n_samples))
if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, idx in enumerate(sample_indices):
    img = X_val[idx]
    mask_true = Y_val[idx]
    mask_pred = Y_val_pred[idx]
    
    # Image
    axes[i, 0].imshow(img if img.ndim == 3 else img, cmap='gray' if img.ndim == 2 else None)
    axes[i, 0].set_title(f"Image {idx}")
    axes[i, 0].axis('off')
    
    # Ground truth
    axes[i, 1].imshow(mask_true, cmap=lbl_cmap)
    n_true = len(np.unique(mask_true)) - 1
    axes[i, 1].set_title(f"Ground Truth ({n_true} objects)")
    axes[i, 1].axis('off')
    
    # Prediction
    axes[i, 2].imshow(mask_pred, cmap=lbl_cmap)
    n_pred = len(np.unique(mask_pred)) - 1
    axes[i, 2].set_title(f"Prediction ({n_pred} objects)")
    axes[i, 2].axis('off')
    
    # Overlay
    axes[i, 3].imshow(img if img.ndim == 3 else img, cmap='gray' if img.ndim == 2 else None)
    axes[i, 3].imshow(mask_pred, cmap=lbl_cmap, alpha=0.5)
    axes[i, 3].set_title("Overlay")
    axes[i, 3].axis('off')

plt.tight_layout()
plt.savefig(f'{model_basedir}/{model_name}/validation_results.png', dpi=150)
plt.show()

print(f"✅ Visualization saved!")

## 15. Save Summary

In [ ]:
summary = f"""
{'='*60}
TRAINING SUMMARY - {model_name}
{'='*60}

📊 DATASET:
   Training images: {n_train}
   Validation images: {n_val}
   Total images: {n_train + n_val}

⚙️ CONFIGURATION:
   n_rays: {conf.n_rays}
   grid: {conf.grid}
   patch_size: {conf.train_patch_size}
   epochs: {conf.train_epochs}
   steps_per_epoch: {conf.train_steps_per_epoch}
   batch_size: {conf.train_batch_size}
   learning_rate: {conf.train_learning_rate}
   GPU: {use_gpu}

🎨 AUGMENTATION:
   Type: Strong (rotation+flip+elastic+brightness+contrast+noise)
   Rotation: 0-360°
   Elastic: α=30, σ=5
   Brightness: ±30%
   Contrast: ±20%

📈 RESULTS:
   AP@0.5: {ap_50:.3f}
   AP@0.6: {stats[1].mean_matched_score:.3f}
   AP@0.7: {stats[2].mean_matched_score:.3f}
   AP@0.8: {stats[3].mean_matched_score:.3f}
   AP@0.9: {stats[4].mean_matched_score:.3f}

🎯 PERFORMANCE:
   {'⭐⭐⭐ EXCELLENT!' if ap_50 >= 0.85 else '⭐⭐ VERY GOOD!' if ap_50 >= 0.75 else '⭐ GOOD' if ap_50 >= 0.65 else '⚠️ NEEDS IMPROVEMENT'}

{'='*60}
"""

print(summary)

# Save to file
with open(f'{model_basedir}/{model_name}/training_summary.txt', 'w') as f:
    f.write(summary)

print(f"✅ Summary saved to: {model_basedir}/{model_name}/training_summary.txt")

## 16. Download Model về máy

In [ ]:
import shutil

# Nén model thành zip
model_path = f"{model_basedir}/{model_name}"
shutil.make_archive(f'{model_name}', 'zip', model_basedir, model_name)

print(f"✅ Model đã được nén: {model_name}.zip")
print(f"\n📥 Download file về máy:")

# Download
from google.colab import files
files.download(f'{model_name}.zip')

print("\n💾 Giải nén file .zip và copy thư mục vào models/ trong project của bạn")

## 17. (Optional) Lưu vào Google Drive

In [ ]:
# Copy model vào Google Drive để lưu trữ lâu dài
drive_save_path = '/content/drive/MyDrive/stardist_models/'
os.makedirs(drive_save_path, exist_ok=True)

shutil.copy2(f'{model_name}.zip', drive_save_path)
print(f"✅ Model đã được lưu vào Google Drive: {drive_save_path}")

---

## 🎉 HOÀN THÀNH!

### Các bước tiếp theo:

1. **Download model về máy** (đã nén thành .zip)
2. **Giải nén** vào thư mục `models/` trong project
3. **Chạy notebook** `2_prediction_my_data.ipynb` để predict trên all frames

### Load model đã train:

```python
from stardist.models import StarDist2D
model = StarDist2D(None, name='stardist_my_data_v2_improved', basedir='models')
```

---

**💡 Tips:**
- Nếu AP < 0.85: Thêm 20-50 ảnh diverse vào training set
- Focus vào các cases khó: overlapping cells, low contrast
- Re-train với dataset mới để improve performance

**Remember**: More high-quality annotations = Better model performance! 🚀